In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [2]:
!wget -q http://apache.osuosl.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!pip install --quiet findspark
!pip install emoji
!pip install language-tool-python
!pip install pysentimiento
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=86eae521d575c4857edefcbf6a869cd37fbae3824cc603c282f33d25649f7508
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [46.8 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,194 kB]
Get:8 https://ppa.launchpadcontent.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark
import pandas as pd
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [6]:
findspark.init()
sc=SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
jdf= spark.read.csv("/content/drive/MyDrive/DATASETS/gaza_israel - gaza_israel.csv", header=True, inferSchema=True)

In [9]:
jdf.show()

+---+----+--------------------+---------------+--------------------+--------------------+--------+--------+---------+-----------+-------------------+--------------------+------------+--------------------+--------+
|_c0| _c1|                Name|       Username|        User Profile|            Tweet ID|Retweets|Comments|Favorites|Is Retweet?|               Date|          Tweet Text|Tweet Source|          Status URL|   SCORE|
+---+----+--------------------+---------------+--------------------+--------------------+--------+--------+---------+-----------+-------------------+--------------------+------------+--------------------+--------+
|  1|NULL|      CNN en Español|          CNNEE|https://www.twitt...|ID: 1719064401330...|      84|     117|      389|       NULL|2023-10-30 18:51:06|"Netanyahu: ""Isr...|        NULL|https://www.twitt...| Neutral|
|  2|NULL|^Malo del Alma^ E...|         jazzun|https://www.twitt...|ID: 1719064401330...|       0|       0|        2|       NULL|2023-10-30 18:5

In [10]:
df = jdf.toPandas()

In [11]:
import re
def cleaned_text(comment):

    caracteres_no_verbales = ['❤', '💔', '😀', '🙂', '👍', '👎', '😂', '✌', '💥', '🌟']

    # Crear una expresión regular para buscar y eliminar estos caracteres
    patron = '|'.join(map(re.escape, caracteres_no_verbales))
    texto_sin_repeticiones = re.sub(r'@[\w]+', '', comment)

    texto_sin_repeticiones = re.sub(patron, '', texto_sin_repeticiones)
    # Expresión regular para buscar y eliminar menciones de usuarios (@nombreusuario)


    # Utilizar una expresión regular para eliminar caracteres no deseados y URLs, pero conservar las letras con tilde, la letra "ñ" y otros caracteres especiales
    texto_sin_repeticiones = re.sub(r'https?://\S+|[^a-zA-Z0-9\s#@áéíóúüñÁÉÍÓÚÜÑ+!]', '', texto_sin_repeticiones)
    texto_sin_repeticiones = re.sub(r'#', '', texto_sin_repeticiones)
    # Convertir el texto a minúsculas antes de la traducción
    texto_sin_repeticiones = texto_sin_repeticiones.lower()

    return texto_sin_repeticiones

In [12]:
df['clean_text'] = df['Tweet Text'].apply(cleaned_text)

In [13]:
from language_tool_python import LanguageTool

# Crear una instancia de LanguageTool para español
tool = LanguageTool('es')

# Función para aplicar correcciones al texto en una columna
def corregir_texto(text):
    return tool.correct(text)

INFO:language_tool_python.download_lt:Unzipping /tmp/tmp_7ibvvo3.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


In [14]:
# Aplicar la corrección a la columna 'texto'
df['correctec_text'] = df['clean_text'].apply(corregir_texto)

In [15]:
df.head()

,_c0,_c1,Name,Username,User Profile,Tweet ID,Retweets,Comments,Favorites,Is Retweet?,Date,Tweet Text,Tweet Source,Status URL,SCORE,clean_text,correctec_text
0,1,None,CNN en Español,CNNEE,https://www.twitter.com/CNNEE,ID: 1719064401330966689,84,117,389,None,2023-10-30 18:51:06,"""Netanyahu: """"Israel no aceptará un cese de ho...",None,https://www.twitter.com/CNNEE/status/171906440...,Neutral,netanyahu israel no aceptará un cese de hostil...,Netanyahu Israel no aceptará un cese de hostil...
1,2,None,^Malo del Alma^ EN CONTRA🇨🇱🌈🪴 😎,jazzun,https://www.twitter.com/jazzun,ID: 1719064401330966689,0,0,2,None,2023-10-30 18:52:11,@CNNEE Fácil decirlo tras las polleras de EEUU,None,https://www.twitter.com/jazzun/status/17190644...,Positivo,fácil decirlo tras las polleras de eeuu,Fácil decirlo tras las polleras de EEUU
2,3,None,Carolina ♡,Carolin72908345,https://www.twitter.com/Carolin72908345,ID: 1719064401330966689,0,0,0,None,2023-10-30 18:53:23,@CNNEE 🫣,None,https://www.twitter.com/Carolin72908345/status...,Neutral,,
3,4,None,Israel Ramos,IsraPanter,https://www.twitter.com/IsraPanter,ID: 1719064401330966689,2,2,7,None,2023-10-30 18:53:31,@CNNEE https://t.co/ph5DRKsGuY,None,https://www.twitter.com/IsraPanter/status/1719...,Negativo,,
4,5,None,Alexis Segovia,AlexisS78626235,https://www.twitter.com/AlexisS78626235,ID: 1719064401330966689,0,0,0,None,2023-10-30 18:54:55,@CNNEE Así es,None,https://www.twitter.com/AlexisS78626235/status...,Neutral,así es,Así es


In [16]:


from pysentimiento.preprocessing import preprocess_tweet
def custom_preprocess(text):
    text= re.sub(r'@\w+', ' ', text)
    text= re.sub(r'https?://\S+|[^a-zA-Z0-9\s#@áéíóúüñÁÉÍÓÚÜÑ+!]', ' ', text)
    # Reemplaza menciones y URLs
    text = preprocess_tweet(text)

    # Acorta caracteres repetidos
    text = preprocess_tweet(text, shorten=2)

    # Normaliza risas
    text = preprocess_tweet(text)

    # Maneja hashtags
    text = preprocess_tweet(text)

    # Maneja emojis
    text = preprocess_tweet(text, lang="es")


    return text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.4 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=77ccc3a3cab7aee38cc1d0a3ef4f3cca483c498a2e9fc2aef83bbb8fd277

In [17]:
df['processed_text'] = df['Tweet Text'].apply(custom_preprocess)

In [18]:
df.head()

,_c0,_c1,Name,Username,User Profile,Tweet ID,Retweets,Comments,Favorites,Is Retweet?,Date,Tweet Text,Tweet Source,Status URL,SCORE,clean_text,correctec_text,processed_text
0,1,None,CNN en Español,CNNEE,https://www.twitter.com/CNNEE,ID: 1719064401330966689,84,117,389,None,2023-10-30 18:51:06,"""Netanyahu: """"Israel no aceptará un cese de ho...",None,https://www.twitter.com/CNNEE/status/171906440...,Neutral,netanyahu israel no aceptará un cese de hostil...,Netanyahu Israel no aceptará un cese de hostil...,Netanyahu Israel no aceptará un cese de hosti...
1,2,None,^Malo del Alma^ EN CONTRA🇨🇱🌈🪴 😎,jazzun,https://www.twitter.com/jazzun,ID: 1719064401330966689,0,0,2,None,2023-10-30 18:52:11,@CNNEE Fácil decirlo tras las polleras de EEUU,None,https://www.twitter.com/jazzun/status/17190644...,Positivo,fácil decirlo tras las polleras de eeuu,Fácil decirlo tras las polleras de EEUU,Fácil decirlo tras las polleras de EEUU
2,3,None,Carolina ♡,Carolin72908345,https://www.twitter.com/Carolin72908345,ID: 1719064401330966689,0,0,0,None,2023-10-30 18:53:23,@CNNEE 🫣,None,https://www.twitter.com/Carolin72908345/status...,Neutral,,,
3,4,None,Israel Ramos,IsraPanter,https://www.twitter.com/IsraPanter,ID: 1719064401330966689,2,2,7,None,2023-10-30 18:53:31,@CNNEE https://t.co/ph5DRKsGuY,None,https://www.twitter.com/IsraPanter/status/1719...,Negativo,,,
4,5,None,Alexis Segovia,AlexisS78626235,https://www.twitter.com/AlexisS78626235,ID: 1719064401330966689,0,0,0,None,2023-10-30 18:54:55,@CNNEE Así es,None,https://www.twitter.com/AlexisS78626235/status...,Neutral,así es,Así es,Así es


In [19]:
#Descargar file comentarios.
from google.colab import files

# Saving the DataFrame to a CSV file
csv_file = 'clean_data.csv'
df.to_csv(csv_file, index=False) # Set index=False if you don't want to include row indices

# Download the file
files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>